In [ ]:
from helper import *
import tensorflow as tf
import numpy as np
import shelve
import joblib
import random
import time

#Cuz the file is inside 'code' directory
mount_point = "../shelved_data/"

with shelve.open(mount_point+'IAM_Data') as shelf:
    vocabulary = shelf['chars']
    list_of_images = shelf['list_of_images']
    image_labels = shelf['image_labels']
    
image_arrays = joblib.load(mount_point+'image_arrays')

/home/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
#List_images ko sort karo
list_of_images.sort()

#Convert vocabulary to list
vocabulary = list(vocabulary)
#Sort so as to have the same ordering every time..
vocabulary.sort()
vocabulary.append("<Blank>")

#Model parameters
hidden_units = 256
num_of_features = 345
seq_len = 2295
batch_size = 256
vocab_size = len(vocabulary)

#Hyper Parameters
alpha = 0.01
epochs = 200
losses = []

# first_image = list_of_images[0]
# label  = image_labels[first_image]
# nparray = image_arrays[first_image]
# one_hot_target = one_hot(label,vocabulary)

In [ ]:
#encode_int(label,vocabulary)

In [ ]:
def sparse_tuple_form(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int32)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int32)

    return indices, values, shape

# CNN-RNN Model

In [5]:
# with tf.device('/gpu:0'):
# Inputs and labels

X = tf.placeholder(tf.float32, [batch_size, seq_len, num_of_features])

targets = tf.sparse_placeholder(tf.int32)

time_steps = np.array([seq_len]*batch_size)

initializer = tf.contrib.layers.xavier_initializer()
# RNN Cell
cell_hidden = tf.contrib.rnn.LSTMCell(hidden_units,num_proj = vocab_size,initializer=initializer)
outputs,states = tf.nn.dynamic_rnn(cell_hidden,X,time_steps,dtype=tf.float32)

# shape = tf.shape(X)
# batch_s , max_time_steps = shape[0], shape[1]

# # Reshaping to apply the same weights over the timesteps
# outputs = tf.reshape(outputs, [-1, hidden_units])

# # Truncated normal with mean 0 and stdev=0.1
# # Tip: Try another initialization
# # see https://www.tensorflow.org/versions/r0.9/api_docs/python/contrib.layers.html#initializers
# W = tf.Variable(tf.truncated_normal([hidden_units,
#                                      vocab_size],
#                                     stddev=0.1))
# # Zero initialization
# # Tip: Is tf.zeros_initializer the same?
# b = tf.Variable(tf.constant(0., shape=[vocab_size]))

# # Doing the affine projection
# logits = tf.matmul(outputs, W) + b

# # Reshaping back to the original shape
# logits = tf.reshape(logits, [batch_s, -1, vocab_size])

# Time major
logits = tf.transpose(outputs, (1, 0, 2))

#Calculate loss
loss = tf.nn.ctc_loss(targets, logits, time_steps)
cost = tf.reduce_mean(loss)

#Optimize
optimizer = tf.train.RMSPropOptimizer(learning_rate=alpha,epsilon=1e-8)
train = optimizer.minimize(loss)

# CTC decoder.

#decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, seq_len)
decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, time_steps)

label_error_rate = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),
                                                   targets))


## Save my MoDel

In [6]:
losses = [] 
outputs = []

saver = tf.train.Saver()

#Take first 52 images
training_list = list_of_images[:128]
# random.seed(100)
# random.shuffle(training_list)

In [7]:
batches_x = []
#Stack images together to form batches
for i in range(0,len(training_list),batch_size):
    #Initialize the current_batch with first image
    current_batch = image_arrays[training_list[i]]
    
    for b in range(1,batch_size):
        #Get the next_image from training_list
        next_image = image_arrays[training_list[i+b]]
        
        #Add it to the current batch, such that the depth increases
        #i.e current_batch --> (height,width,1), next_image --> (height,width)
        # then current_batch --> (height,width,2) and so, on until you get
        #current_batch --> (height,width,batch_size)
        current_batch = np.dstack((current_batch,next_image))
    batches_x.append(current_batch)

batches_y = []

for i in range(0,len(training_list),batch_size):
    #Get the current batch, list of sequences(lists)
    current_batch = [encode_int(image_labels[training_list[i+b]],vocabulary) for b in range(batch_size)]
    
    #Convert the list of sequences(lists) to sparse_tuple_form and add them to batches_y
    batches_y.append(sparse_tuple_form(current_batch))

print(len(batches_x),len(batches_y))

1 1


In [8]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs): 
        start_time = time.time()
        total_cost,total_ler = 0.0,0.0
        
        #Iterate through all images in a single epoch...
        for b in range(len(batches_x)):
            #print(b,training_list[b])
            
            #Before feeding x reshape it as (batch_size,width,height)
            feed = {X:np.transpose(batches_x[b],(2,1,0)),targets:batches_y[b]}

            sess.run(train,feed_dict=feed)
            
        if e % 1 == 0:
            cost_val,ler_val,d = sess.run([cost,label_error_rate,decoded], feed_dict=feed)
            #print("Epoch {0}: cost = {1} ler = {2}".format(e,cost_val,ler_val))
            total_cost+=cost_val
            total_ler+=ler_val
            
            losses.append(total_cost)
            if e % 1 == 0:
                outputs.append(d)
        end_time = time.time()
        time_taken = end_time - start_time
        
        print("Epoch {}: cost = {} ler = {:.2f} - Time taken:{:.2f} sec".format(e,total_cost,total_ler,time_taken))
        
        if e%10==0:
            saver.save(sess,'../model/Lines_RNN_'+str(e))

# plt.plot(list(range(len(losses))),losses)
# plt.xlabel('Epochs')
# plt.ylabel('Loss')

Epoch 0: cost = 1558.856201171875 ler = 1.00 - Time taken:17.07 sec
Epoch 1: cost = 186.9901123046875 ler = 1.00 - Time taken:16.33 sec
Epoch 2: cost = 166.76568603515625 ler = 1.00 - Time taken:16.34 sec
Epoch 3: cost = 158.72933959960938 ler = 1.00 - Time taken:16.48 sec
Epoch 4: cost = 153.25625610351562 ler = 1.00 - Time taken:16.28 sec
Epoch 5: cost = 150.19598388671875 ler = 1.00 - Time taken:16.35 sec
Epoch 6: cost = 148.00119018554688 ler = 1.00 - Time taken:16.45 sec
Epoch 7: cost = 146.8482208251953 ler = 1.00 - Time taken:16.28 sec


KeyboardInterrupt: 

## Predict using Model

In [ ]:
# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('Basic_RNN_TF_API.ckpt.meta')
#     saver.restore(sess, '../Lines_RNN.ckpt')

#     X = sess.graph.get_tensor_by_name('Placeholder:0')
    
#     for _ in range(30): 
#         y_pred = sess.run('rnn/transpose:0', feed_dict={X: X_new})
#         #print(y_pred)
#         #print(y_pred.shape)
#         content += interpret(y_pred,vocabulary)
#         X_new = y_pred
        
# print(content)

In [ ]:
#fig=plt.figure()
#for i in range(len(training_list)):
    #fig.add_subplot(i,2,1)
# plt.imshow(image_arrays[training_list[0]],cmap="gray")

In [ ]:
content = []

In [ ]:
for k in range(0,5):
    val = outputs[k][0]
    content.append(''.join([vocabulary[x] for x in np.asarray(val[1])]))

In [ ]:
print("\n".join(content))